In [1]:
#connect to colab
!git clone https://github.com/jkohler-u/Denoise_tf.git
import sys
sys.path.append('/content/Denoise_tf')

Cloning into 'Denoise_tf'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 59 (delta 29), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), 6.02 MiB | 2.52 MiB/s, done.


In [2]:
#our own pyfiles
import preprocessing
import unet
import postprocessing

In [3]:
#load tensorboard
%load_ext tensorboard

In [4]:
#working with files
from zipfile import ZipFile
from google.colab import drive
import os

filename = 'a1'

# Mount your Google Drive
drive.mount('/content/drive')

# Change the working directory to your Drive
os.chdir('/content/drive/MyDrive')

# Open the zipfile and extract its contents to memory
with ZipFile('CleanSpeech_training.zip', 'r') as zip_file:
  zip_file.extractall(filename)

filename = 'a2'

# Open the zipfile and extract its contents to memory
with ZipFile('NoisySpeech_training.zip', 'r') as zip_file:
  zip_file.extractall(filename)

Mounted at /content/drive


In [5]:
#creating the dataset
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

#load the data 
train_noisy = preprocessing.prepare_data('a2')
train_clean = preprocessing.prepare_data('a1')
train_noisy = preprocessing.make_same(train_noisy, 196500)
train_clean = preprocessing.make_same(train_clean, 196500)

# Turn all the data into complex spectrograms
clean_spec = preprocessing.spectrogram(np.array(train_clean))
noisy_spec = preprocessing.spectrogram(np.array(train_noisy))

#convert the data into a tf dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((noisy_spec, clean_spec))

# Calculate the total number of samples in the dataset
num_samples = tf.data.experimental.cardinality(tf_dataset).numpy()

# Shuffle the dataset
tf_dataset = tf_dataset.shuffle(num_samples)

#randomly split the dataset into train and test sets
train_size = int(0.8 * num_samples)
test_size = num_samples - train_size
train_dataset = tf_dataset.take(train_size)
test_dataset = tf_dataset.skip(train_size)

#batch
batch_size = 10
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

/content/Denoise_tf/preprocessing.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [ ]:
'''fit function'''
#tf
import datetime
import tensorflow as tf
from tensorflow import keras

#added tensorboard log
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#run the model
unet.model.fit(
    train_dataset,
    epochs=20,
    shuffle=True,
    validation_data=test_dataset,
    callbacks=[tensorboard_callback],
)

Epoch 1/20
10/10 [==============================] - 65s 4s/step - loss: 0.3840 - val_loss: 0.2750
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.2682

In [ ]:
# Convert tf dataset back to numpy arrays
prediciton, noisy, clean = postprocessing.get_one_of_each(train_dataset)

#prediction_for_rest = prediction_for_rest.take(1)
pred = (unet.model.predict(prediciton.batch(1)))
pred = tf.squeeze(pred)

In [ ]:
#plotting the data
import matplotlib.pyplot as plt

f, axarr = plt.subplots(1,3, figsize=(20,4))
axarr[0].imshow(noisy, aspect='auto', origin='lower', cmap='viridis')
axarr[0].set_title("noisy")
axarr[1].imshow(clean, aspect='auto', origin='lower', cmap='viridis')
axarr[1].set_title("clean")
axarr[2].imshow(pred, aspect='auto', origin='lower', cmap='viridis')
axarr[2].set_title("prediction")


In [ ]:
#displying audio
import IPython
import soundfile as sf

#convert the spectograms back to audio
n = postprocessing.convert_to_audio(noisy)
c = postprocessing.convert_to_audio(clean)
p = postprocessing.convert_to_audio(pred)

#create audio files
sr = 16000
sf.write('noisy.wav', n, sr)
sf.write('clean.wav', c, sr)
sf.write('pred.wav', p, sr)

#noisy audio
IPython.display.Audio('noisy.wav')

In [ ]:
#clean audio
IPython.display.Audio('clean.wav')

In [ ]:
#predicted audio
IPython.display.Audio('pred.wav')

In [ ]:
%tensorboard --logdir logs/fit